In [ ]:
#import required libraries
import pandas as pd
import numpy as np
import requests as req
from bs4 import BeautifulSoup
import nltk
nltk.download('all')
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from textblob import TextBlob
!pip install textstat
import textstat
from textstat.textstat import textstatistics

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Load given Input data
d_frame=pd.read_csv('/content/Input.xlsx - Sheet1.csv')
df=d_frame.copy() 
df.head()

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...


Collect stop words from given text files

In [ ]:
f1 = open(r"/content/StopWords_Currencies.txt","r",encoding="ISO-8859-1")
f2 = open(r"/content/StopWords_DatesandNumbers.txt","r",encoding="ISO-8859-1")
f3 = open(r"/content/StopWords_Generic.txt","r",encoding="ISO-8859-1")
f4 = open(r"/content/StopWords_GenericLong.txt","r",encoding="ISO-8859-1")
f5 = open(r"/content/StopWords_Geographic.txt","r",encoding="ISO-8859-1")
f6 = open(r"/content/StopWords_Names.txt","r",encoding="ISO-8859-1")
l1 = f1.readlines()
l2 = f2.readlines()
l3 = f3.readlines()
l4 = f4.readlines()
l5 = f5.readlines()
l6 = f6.readlines()
def read(l):
  return f1.readlines()
stop1=[i.split('|')[0].strip() for i in l1 ]
stop2=[i.strip() for i in l2]
stop3=[i.strip() for i in l3]
stop4=[i.strip() for i in l4]
stop5=[i.strip() for i in l5]
stop6=[i.strip() for i in l6]
stop=stop1+stop2+stop3+stop4+stop5+stop6

Extract text using BeautifulSoup

In [ ]:
#create function for extract required text
def gettext(link):
  response=req.get(link)
  data=BeautifulSoup(response.text,'html.parser')
  text=data.head.title.text
  for para in data.find_all("p"):
      text=text+para.get_text()
  return text

In [ ]:
df['text']=df.URL.apply(gettext)
#Required text stored as text feature in dataframe
df

,URL_ID,URL,text
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes -...
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creato...
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us? - Blackcof...
...,...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...,Blockchain for Payments - Blackcoffer Insights...
110,147,https://insights.blackcoffer.com/the-future-of...,The future of Investing - Blackcoffer Insights...
111,148,https://insights.blackcoffer.com/big-data-anal...,Big Data Analytics in Healthcare - Blackcoffer...
112,149,https://insights.blackcoffer.com/business-anal...,Business Analytics In The Healthcare Industry ...


Lemmatization(with pos) and Removed stop Words

In [ ]:
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet

# Create WordNetLemmatizer object
lemmatizer = WordNetLemmatizer()

punctuations = list(string.punctuation)
punctuations.remove('.')
stops = stop + punctuations
# defining a func for simple_pos_tag

def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    
    elif tag.startswith('V'):
        return wordnet.VERB
    
    elif tag.startswith('N'):
        return wordnet.NOUN
    
    elif tag.startswith('R'):
        return wordnet.ADV
    
    else:
        return wordnet.NOUN

#convert list to string
def listToString(s):
   
    # initialize an empty string
    str1 = " "
    return (str1.join(s))

def clean_process(words):
    words=word_tokenize(words)
    output_words = []
    for w in words:
        if w.lower() not in stops:
            pos = pos_tag([w])
            clean_words = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_words.lower())
            
    final_words=listToString(output_words)
    return final_words


In [ ]:
df['clean_text']=df.text.apply(clean_process)
#clean text stored as clean_text in df

In [ ]:
df

,URL_ID,URL,text,clean_text
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes -...,ai healthcare improve patient outcomes blackco...
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creato...,creation taking creator blackcoffer insightshu...
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,jobs robots humans future blackcoffer insights...
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,machine replace human future work blackcoffer ...
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us? - Blackcof...,ai replace work blackcoffer insights “ machine...
...,...,...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...,Blockchain for Payments - Blackcoffer Insights...,blockchain payments blackcoffer insightsreconc...
110,147,https://insights.blackcoffer.com/the-future-of...,The future of Investing - Blackcoffer Insights...,future investing blackcoffer insightsan invest...
111,148,https://insights.blackcoffer.com/big-data-anal...,Big Data Analytics in Healthcare - Blackcoffer...,big data analytics healthcare blackcoffer insi...
112,149,https://insights.blackcoffer.com/business-anal...,Business Analytics In The Healthcare Industry ...,business analytics healthcare industry blackco...


collect positive and negative words from given files 

In [ ]:
n = open(r"/content/negative-words.txt","r",encoding="ISO-8859-1")
p = open(r"/content/positive-words.txt","r",encoding="ISO-8859-1")
positive=p.readlines()
positive=[i.strip() for i in positive]
negative=n.readlines()
negative=[i.strip() for i in negative]

Create dictionary of positive and negative words

In [ ]:
dict_positive={}
dict_negative={}
for para in df.clean_text:
  t=word_tokenize(para)
  for word in t:
    if word.lower().strip() in positive:
      dict_positive[word]=dict_positive.get(word,0)+1
    if word.lower().strip() in negative:
      dict_negative[word]=dict_negative.get(word,0)+1


create function for each required variables

In [ ]:

#function for getting positive score
def positive(text):
  fin_positive={}
  t=word_tokenize(text)
  for word in t:
    if word.lower().strip() in dict_positive.keys():
      fin_positive[word]=fin_positive.get(word,0)+1
  return sum(fin_positive.values())

#function for getting negative score
def negative(text):
  fin_negative={}
  t=word_tokenize(text)
  for word in t:
    if word.lower().strip() in dict_negative.keys():
      fin_negative[word]=fin_negative.get(word,0)-1
  return sum(fin_negative.values())*(-1)

#function for getting polarity
def getPolarity(text):
  pos_score=positive(text)
  neg_score=negative(text)
  return round((pos_score-neg_score)/((pos_score+neg_score)+0.000001),3)

#function for getting word count
def word_count(text):
  count=0
  t=word_tokenize(text)
  for i in t:
    count+=1
  return count

#function for getting sentence count
def sentence_count(text):
  sen=sent_tokenize(text)
  return len(sen)

#function for getting subjectivity
def getSubjectivity(text):
  pos_score=positive(text)
  neg_score=negative(text)
  t_word=word_count(text)
  ans=(pos_score+neg_score)/((t_word)+0.000001)
  return round(ans,3)

#function for getting syllables count
def syllables_count(word):
    ans=textstatistics().syllable_count(word)
    return round(ans,3)

#function for getting complex word count
def complex_words(text):

    words = []
    word_all=word_tokenize(text)
    words += [str(token) for token in word_all]
    # difficult words are those with syllables > 2
    diff_words_set = set()
     
    for word in words:
        syllable_count = syllables_count(word)
        if syllable_count > 2:
            diff_words_set.add(word)
 
    return len(diff_words_set)

#function for getting percentage of complex words
def percen_of_complex_word(text):
  complex_word=complex_words(text)
  total_word=word_count(text)
  ans=(complex_word/total_word)*100
  return round(ans,3)

#function for average word length
def avg_word_length(text):
  wordd_count=word_count(text)
  ans= len([i.split() for i in text])/wordd_count
  return round(ans,3)

##function for getting average sentence length
def avg_sen_length(text):
  w_count=word_count(text)
  s_count=sentence_count(text)
  return round(w_count/s_count,3)

#function for getting syllables per word
def syllables_per_word(text):
    syllable = syllables_count(text)
    words = word_count(text)
    ans = float(syllable) / float(words)
    return round(ans,3)      

#function for getting average number of word per sentence
def avg_num_of_word_per_sen(text):
  w_countt=word_count(text)
  s_count=sentence_count(text)
  ans=w_countt/s_count 
  return round(ans,3)

#function for getting personal pronoun
def personal_pronoun(text):
  l=['i','we','my','ours','us']
  count=0
  for i in list(text):
    if i.lower() in l:
      count+=1
  return count

#function for getting fox index
def fog_index(text):                          
  ans=0.4*(avg_sen_length(text)+percen_of_complex_word(text))
  return round(ans,3)



Using APPLY function extract required variables

In [ ]:
df['Positive SCORE']=df['clean_text'].apply(positive)
df['Negative SCORE']=df['clean_text'].apply(negative)
df['POLARITY SCORE']=df['clean_text'].apply(getPolarity)
df['SUBJECTIVITY SCORE']=df['clean_text'].apply(getSubjectivity)
df['AVG SENTENCE LENGTH']=df.clean_text.apply(avg_sen_length)
df['PERCENTAGE OF COMPLEX WORDS']=df.clean_text.apply(percen_of_complex_word)
df['FOG INDEX']=df.clean_text.apply(fog_index)
df['AVG NUMBER OF WORDS PER SENTENCE']=df.clean_text.apply(avg_num_of_word_per_sen)
df['COMPLEX WORD COUNT']=df.clean_text.apply(complex_words)
df['WORD COUNT']=df.clean_text.apply(word_count)
df['SYLLABLE PER WORD']=df.clean_text.apply(syllables_per_word)
df['PERSONAL PRONOUNS']=df.clean_text.apply(personal_pronoun)
df['AVG WORD LENGTH']=df.clean_text.apply(avg_word_length)


In [ ]:
df

,URL_ID,URL,text,clean_text,Positive SCORE,Negative SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,AI in healthcare to Improve Patient Outcomes -...,ai healthcare improve patient outcomes blackco...,73,32,0.390,0.097,19.618,21.409,16.411,19.618,231,1079,2.076,633,7.816
1,38,https://insights.blackcoffer.com/what-if-the-c...,What if the Creation is Taking Over the Creato...,creation taking creator blackcoffer insightshu...,74,38,0.321,0.161,12.393,18.012,12.162,12.393,125,694,1.775,350,6.866
2,39,https://insights.blackcoffer.com/what-jobs-wil...,What Jobs Will Robots Take From Humans in The ...,jobs robots humans future blackcoffer insights...,65,42,0.215,0.114,13.970,21.368,14.135,13.970,200,936,2.058,546,7.691
3,40,https://insights.blackcoffer.com/will-machine-...,Will Machine Replace The Human in the Future o...,machine replace human future work blackcoffer ...,66,27,0.419,0.122,11.418,16.078,10.998,11.418,123,765,1.822,420,7.014
4,41,https://insights.blackcoffer.com/will-ai-repla...,Will AI Replace Us or Work With Us? - Blackcof...,ai replace work blackcoffer insights “ machine...,63,25,0.432,0.097,13.567,15.402,11.588,13.567,140,909,1.851,430,7.068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...,Blockchain for Payments - Blackcoffer Insights...,blockchain payments blackcoffer insightsreconc...,26,29,-0.055,0.112,12.974,24.746,15.088,12.974,122,493,1.947,243,7.803
110,147,https://insights.blackcoffer.com/the-future-of...,The future of Investing - Blackcoffer Insights...,future investing blackcoffer insightsan invest...,37,14,0.451,0.082,16.757,18.387,14.058,16.757,114,620,1.916,308,7.421
111,148,https://insights.blackcoffer.com/big-data-anal...,Big Data Analytics in Healthcare - Blackcoffer...,big data analytics healthcare blackcoffer insi...,39,47,-0.093,0.129,10.918,15.616,10.614,10.918,104,666,1.872,353,7.071
112,149,https://insights.blackcoffer.com/business-anal...,Business Analytics In The Healthcare Industry ...,business analytics healthcare industry blackco...,36,4,0.800,0.132,18.938,24.422,17.344,18.938,74,303,2.244,202,8.191


In [ ]:
Blackcoffer=df.drop(['text','clean_text'],axis=1)
Blackcoffer.to_csv('Blackcoffer.csv', index=False)
